# Options for K$_z$

In this tutorial you will learn: 

1. The hierarchy of K$_z$ assumptions 

You should already be familiar with: 

1. How to pick condensates to run in a cloud model 
2. What is $f_{sed}$ and what number is right for my model? 
3. What are the chemical limitations in the code 
4. How to compute initial Mie scattering grid
5. Generally how to run the code 


In [ ]:
from bokeh.io import output_notebook 
from bokeh.plotting import show, figure
output_notebook()
import numpy as np
import pandas as pd
import astropy.units as u

#cloud code
import virga.justdoit as vdi
import virga.justplotit as vpi


Initial parameters we will keep fixed for the tutorial

In [ ]:
mieff_directory =  '/Users/nbatalh1/Documents/data/virga'
fsed = 0.3
metallicity = 1 #atmospheric metallicity relative to Solar
mean_molecular_weight = 2.2 # atmospheric mean molecular weight

## Toy Model: Supply Constant K$_z$

In [ ]:
sum_planet = vdi.Atmosphere(['KCl', 'MgSiO3'],fsed=fsed,mh=metallicity,
                 mmw = mean_molecular_weight)
sum_planet.gravity(gravity=100.00, gravity_unit=u.Unit('m/(s**2)'))

In [ ]:
constant_kz = 1e10
df = vdi.brown_dwarf()
sum_planet.ptk(df = df.loc[:,['pressure','temperature']],constant_kz = constant_kz) 

In [ ]:
out_constant = sum_planet.compute(as_dict=True, 
                               directory=mieff_directory)

## Informed: Compute K$_z$ from convective heat flux 

The convective heat flux is supplied through an external radiative transfer model which partitions the transport of interior heat between radiative and convective fluxes. As an example, we will upload one from the Sonora Grid that has a column `chf` which stands for "Convective Heat Flux" in units of `ergs/cm2/s`

In [ ]:
df = vdi.brown_dwarf()
sum_planet = vdi.Atmosphere(['MgSiO3','KCl'],fsed=fsed,mh=metallicity,
                 mmw = mean_molecular_weight)
sum_planet.gravity(gravity=100.00, gravity_unit=u.Unit('m/(s**2)'))
sum_planet.ptk(df=df)
out_chf = sum_planet.compute(as_dict=True, 
                               directory=mieff_directory)

### Turn on correction for latent heat

In [ ]:
sum_planet = vdi.Atmosphere(['MgSiO3','KCl'],fsed=fsed,mh=metallicity,
                 mmw = mean_molecular_weight)
sum_planet.gravity(gravity=100.00, gravity_unit=u.Unit('m/(s**2)'))
sum_planet.ptk(df = df, latent_heat=True)
out_latent = sum_planet.compute(as_dict=True, 
                               directory=mieff_directory)

### Turn on correction for latent heat & convective overshoot

In [ ]:
sum_planet = vdi.Atmosphere(['MgSiO3','KCl'],fsed=fsed,mh=metallicity,
                 mmw = mean_molecular_weight)
sum_planet.gravity(gravity=100.00, gravity_unit=u.Unit('m/(s**2)'))
sum_planet.ptk(df = df, latent_heat=True, convective_overshoot=1/3.)
out_latent_cos = sum_planet.compute(as_dict=True, 
                               directory=mieff_directory)

## Analyze different $K_z$ formalism via convective heat flux

In [ ]:
legend = ['Constant','CHF-Only','CHF+Latent','CFH+Latent+COS']
fig =vpi.pressure_fig(plot_height=250,x_axis_label='Kz cm2/s',x_axis_type='log',x_range=[1e8,5e11])
for i,idf in enumerate([out_constant, out_chf, out_latent, out_latent_cos ]):
    fig.line(idf['kz'],idf['pressure'],line_width=3,color=vpi.colpals.Colorblind8[i],legend_label=legend[i])
show(fig)

In [ ]:
show(vpi.all_optics_1d([out_constant, out_chf, out_latent, out_latent_cos ], wave_range=[0.3,1],
                       legend = legend))

## More informed: Input K$_z$ from external model 

(e.g. Global Circulation Model) 

In [ ]:
df = vdi.hot_jupiter()

sum_planet = vdi.Atmosphere(['MgSiO3','KCl'],fsed=1,mh=metallicity,
                 mmw = mean_molecular_weight)
sum_planet.gravity(gravity=25.00, gravity_unit=u.Unit('m/(s**2)'))
sum_planet.ptk(df=df)
hj_real = sum_planet.compute(directory=mieff_directory,as_dict=True)
sum_planet.ptk(df=df.loc[:,['pressure','temperature']],constant_kz=1e10)
hj_constant = sum_planet.compute(directory=mieff_directory,as_dict=True)

## Analyze different $K_z$ formalism via external input


In [ ]:
#get full dictionary output 
show(vpi.all_optics_1d([hj_real,hj_constant],[0.3,1] , legend=['Real','Constant']))